# Acquiring and processing information on world's largest banks

## Importing libraries and files

In [1]:
import numpy as np
import pandas as pd
import requests
import sqlite3
from bs4 import BeautifulSoup
from datetime import datetime

# Get exchange_rate.csv file by running the following command in the terminal:
# wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv

## Functions needed

In [9]:
# Code for ETL operations on Country-GDP data

# Importing the required libraries

def log_progress(message):
    ''' This function logs the mentioned message of a given stage of the
    code execution to a log file. Function returns nothing'''
    
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second 
    now = datetime.now() # get current timestamp 
    timestamp = now.strftime(timestamp_format) 
    with open("./code_log.txt","a") as f: 
        f.write(timestamp + ' : ' + message + '\n')

def extract(url, table_attribs):
    ''' This function aims to extract the required
    information from the website and save it to a data frame. The
    function returns the data frame for further processing. '''
    page = requests.get(url).text
    data = BeautifulSoup(page, 'html.parser')
    df = pd.DataFrame(columns=table_attribs)
    tables = data.find_all('tbody')
    rows = tables[0].find_all('tr')
    for row in rows:
        col = row.find_all('td')
        if (len(col)==0):
            continue
        if (col[1].find('a')==None):
            continue
        data_dict = {"Name": col[1].find_all('a')[1].contents[0],
                     "MC_USD_Billion": float(col[2].contents[0].replace('\n', ''))}
        df1 = pd.DataFrame(data_dict, index=[0])
        if df.empty:
            df = df1.copy()
        else:
            df = pd.concat([df, df1], ignore_index=True)
    return df

def transform(df, csv_path):
    ''' This function accesses the CSV file for exchange rate
    information, and adds three columns to the data frame, each
    containing the transformed version of Market Cap column to
    respective currencies'''
    exchange_rate = pd.read_csv(csv_path).set_index('Currency').to_dict()['Rate']
    MC_USD_list = df['MC_USD_Billion'].tolist()
    for key, val in exchange_rate.items():
        df[f'MC_{key}_Billion'] = list(map(lambda x: np.round(x*val, 2), MC_USD_list))
    return df

def load_to_csv(df, output_path):
    ''' This function saves the final data frame as a CSV file in
    the provided path. Function returns nothing.'''
    df.to_csv(output_path)

def load_to_db(df, sql_connection, table_name):
    ''' This function saves the final data frame to a database
    table with the provided name. Function returns nothing.'''
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)

def run_query(query_statement, sql_connection):
    ''' This function runs the query on the database table and
    prints the output on the terminal. Function returns nothing. '''
    print(query_statement)
    query_output = pd.read_sql(query_statement, sql_connection)
    return(query_output)

## Main code

### ETL Process

In [7]:
''' Here, you define the required entities and call the relevant
functions in the correct order to complete the project. Note that this
portion is not inside any function.'''

url = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'
table_attribs = ['Name', 'MC_USD_Billion']
exchange_csv_path = 'exchange_rate.csv'
output_csv_path = 'Largest_banks_data.csv'
table_name = 'Largest_banks'

log_progress('Preliminaries complete. Initiating ETL process')

df = extract(url, table_attribs)

log_progress('Data extraction complete. Initiating Transformation process')

df = transform(df, exchange_csv_path )

log_progress('Data transformation complete. Initiating loading process')

load_to_csv(df, output_csv_path)

log_progress('Data saved to CSV file')

sql_connection = sqlite3.connect('Banks.db')

log_progress('SQL Connection initiated.')

load_to_db(df, sql_connection, table_name)

log_progress('Data loaded to Database as  table. Running the query')

df

,Name,MC_USD_Billion,MC_EUR_Billion,MC_GBP_Billion,MC_INR_Billion
0,JPMorgan Chase,432.92,402.62,346.34,35910.71
1,Bank of America,231.52,215.31,185.22,19204.58
2,Industrial and Commercial Bank of China,194.56,180.94,155.65,16138.75
3,Agricultural Bank of China,160.68,149.43,128.54,13328.41
4,HDFC Bank,157.91,146.86,126.33,13098.63
5,Wells Fargo,155.87,144.96,124.70,12929.42
6,HSBC Holdings PLC,148.90,138.48,119.12,12351.26
7,Morgan Stanley,140.83,130.97,112.66,11681.85
8,China Construction Bank,139.82,130.03,111.86,11598.07
9,Bank of China,136.81,127.23,109.45,11348.39


### Query 1

In [10]:
query_statement_1 = f'SELECT * FROM {table_name}'
run_query(query_statement_1, sql_connection)

SELECT * FROM Largest_banks


,Name,MC_USD_Billion,MC_EUR_Billion,MC_GBP_Billion,MC_INR_Billion
0,JPMorgan Chase,432.92,402.62,346.34,35910.71
1,Bank of America,231.52,215.31,185.22,19204.58
2,Industrial and Commercial Bank of China,194.56,180.94,155.65,16138.75
3,Agricultural Bank of China,160.68,149.43,128.54,13328.41
4,HDFC Bank,157.91,146.86,126.33,13098.63
5,Wells Fargo,155.87,144.96,124.70,12929.42
6,HSBC Holdings PLC,148.90,138.48,119.12,12351.26
7,Morgan Stanley,140.83,130.97,112.66,11681.85
8,China Construction Bank,139.82,130.03,111.86,11598.07
9,Bank of China,136.81,127.23,109.45,11348.39


### Query 2

In [11]:
query_statement_2 = f'SELECT AVG(MC_GBP_Billion) FROM {table_name}'
run_query(query_statement_2, sql_connection)

SELECT AVG(MC_GBP_Billion) FROM Largest_banks


,AVG(MC_GBP_Billion)
0,151.987


### Query 3

In [12]:
query_statement_3 = f'SELECT Name from {table_name} LIMIT 5'
run_query(query_statement_3, sql_connection)

SELECT Name from Largest_banks LIMIT 5


,Name
0,JPMorgan Chase
1,Bank of America
2,Industrial and Commercial Bank of China
3,Agricultural Bank of China
4,HDFC Bank


### Closing connection to SQL database

In [13]:
log_progress('Process Complete.')

sql_connection.close()